In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.magic import register_cell_magic

from y2klc3tools import PC_START, UINT16_MAX
from y2klc3tools.sqlvm import SqlVM

In [ ]:
vm = SqlVM()


@register_cell_magic
def sql(_line: str, cell: str) -> None:
    vm.run_and_trace(cell)

In [ ]:
vm.mem.load_binary_from_file("../obj/vm/2048.obj")

In [ ]:
from y2klc3tools.asm import assemble

code = """
.ORIG x3000
LEA R0, HELLO_WORLD
PUTS
HALT
HELLO_WORLD .stringz "Hello, World"
.END
"""

sym, image = assemble(code)
print(image)
vm.mem.load_binary_from_bytes(image)
vm.reset()

In [ ]:
vm.step()
vm.run_and_trace("""
    SELECT * FROM signal;
    SELECT * FROM register;
    SELECT * FROM memory
        WHERE address >= 0x3000 and address < 0x3040;
    """)

In [ ]:
%%sql
SELECT * FROM signal;
SELECT * FROM register;
SELECT * FROM memory
    WHERE address >= 0x3000 and address < 0x3040;

In [ ]:
origin = '0000'
image_bytes = origin + '16BF' * (UINT16_MAX)

vm.reset()
vm.mem.load_binary_from_hex(image_bytes)

vm.mem[PC_START + 0x10] = 0xF025  # HLT

In [ ]:
vm.step()
vm.run_and_trace(
    """
    SELECT * FROM signal;
    SELECT * FROM register;
    SELECT * FROM memory
        WHERE address >= 0x3000 and address < 0x3040;
    """
)

In [ ]:
import ipywidgets as widgets
from IPython.display import display


class MyCat:
    def __init__(self, name: str):
        self.name = name
        self.widget = widgets.HTML()

    def update_name(self, name: str) -> None:
        self.name = name
        self._ipython_display_update()

    def _ipython_display_update(self):
        self.widget.value = f"<h1>{self.name}</h1>"

    def _ipython_display_(self):
        self._ipython_display_update()
        display(self.widget)

In [ ]:
kitt = MyCat("Fluffy")

In [ ]:
kitt

In [ ]:
kitt.update_name("Whiskers")